## AML Chat Targets

This code shows how to use Azure Machine Learning (AML) managed online endpoints with PyRIT.

### Prerequisites

1. **Deploy an AML-Managed Online Endpoint:** Confirm that an Azure Machine Learning managed online endpoint is
     already deployed.

1. **Obtain the API Key:**
   - Navigate to the AML Studio.
   - Go to the 'Endpoints' section.
   - Retrieve the API key and endpoint URI from the 'Consume' tab
   <br> <img src="../../../assets/aml_managed_online_endpoint_api_key.png" alt="aml_managed_online_endpoint_api_key.png" height="400"/> <br>

1. **Set the Environment Variable:**
   - Add the obtained API key to an environment variable named `AZURE_ML_KEY`. This is the default API key when the target is instantiated.
   - Add the obtained endpoint URI to an environment variable named `AZURE_ML_MANAGED_ENDPOINT`. This is the default endpoint URI when the target is instantiated.
   - If you'd like, feel free to make additional API key and endpoint URI environment variables in your .env file for different deployed models (e.g. Llama, Phi-3, Mistral, etc.)
     and pass them in as arguments to the `_set_env_configuration_vars` function to interact with those models.
   

### Create a AzureMLChatTarget

After deploying a model and populating your env file, send prompts to the model using the `AzureMLChatTarget` class. Model parameters can be passed upon instantiation
or set using the _set_model_parameters() function. `**param_kwargs` allows for the setting of other parameters not explicitly shown in the constructor. A general list of
possible adjustable parameters can be found here: https://huggingface.co/docs/api-inference/tasks/text-generation but note that not all parameters may have an effect
depending on the specific model. The parameters that can be set per model can usually be found in the 'Consume' tab when you navigate to your endpoint in AML Studio.

In [3]:
from pyrit.common import default_values
from pyrit.orchestrator import PromptSendingOrchestrator
from pyrit.prompt_target import AzureMLChatTarget

default_values.load_default_env()

# Defaults to endpoint and api_key pulled from the AZURE_ML_MANAGED_ENDPOINT and AZURE_ML_KEY environment variables
azure_ml_chat_target = AzureMLChatTarget()
# The environment variable args can be adjusted below as needed for your specific model.
azure_ml_chat_target._set_env_configuration_vars(endpoint_uri_environment_variable="AZURE_ML_MANAGED_ENDPOINT", api_key_environment_variable="AZURE_ML_KEY")
# Parameters such as temperature and repetition_penalty can be set using the _set_model_parameters() function. 
azure_ml_chat_target._set_model_parameters(temperature = 0.9, repetition_penalty=1.3)

with PromptSendingOrchestrator(prompt_target=azure_ml_chat_target) as orchestrator:
    response = await orchestrator.send_prompts_async(prompt_list=["Hello! Describe yourself and the company who developed you."])  # type: ignore
await orchestrator.print_conversations() # type: ignore

Conversation ID: dea28454-dbf6-4f39-95e6-763bfe1b29ff
user: Hello! Describe yourself and the company who developed you.
assistant: Hello! I'm an assistant developed by Mistral AI, a leading AI company based in Paris, France. Mistral AI is dedicated to building large language models that can understand and generate human-like text. I am designed to assist with a wide range of tasks, including answering questions, generating text, providing explanations, and much more.

At Mistral AI, we believe that AI has the potential to revolutionize the way we live and work. Our team of world-class researchers and engineers are committed to building AI systems that are safe, transparent, and beneficial to society. We are proud to be part of the vibrant AI ecosystem in France and are dedicated to contributing to the global AI research community.

As for me, I am constantly learning and improving, and I'm excited to help you with any questions or tasks you may have!



You can then use this cell anywhere you would use a `PromptTarget` object.
For example, you can create a red teaming orchestrator and use this instead of the `AzureOpenAI` target and do the [Gandalf or Crucible Demos](./3_custom_targets.ipynb) but use this AML model.

This is also shown in the [Red Teaming Orchestrator](../orchestrators/3_red_teaming_orchestrator.ipynb) documentation.